<a href="https://colab.research.google.com/github/hoonzi-s/hongong_MLDL/blob/main/05-2.%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증 세트

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split as tts
train_input, test_input, train_target, test_target = tts(data, target, test_size = 0.2, random_state = 42)

In [4]:
sub_input, val_input, sub_target, val_target = tts(train_input, train_target, test_size = 0.2, random_state = 42)

In [5]:
print(data.shape, train_input.shape, sub_input.shape, val_input.shape, test_input.shape)

(6497, 3) (5197, 3) (4157, 3) (1040, 3) (1300, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증

In [7]:
# k-폴드 교차 검증
# 보통 5-폴드, 10-폴드 교차 검증을 많이 사용
# 데이터의 80~90%까지 훈련에 사용 가능

In [8]:
# cross_validate() 교차 검증 함수 (기본적으로 5-폴드)
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.02180767, 0.02462482, 0.05100203, 0.01523638, 0.02298355]),
 'score_time': array([0.00128388, 0.00118852, 0.00759387, 0.00130129, 0.0124886 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}